# Test `AnnDataAccessor`

In [ ]:
import lamindb as ln

ln.setup.init(storage="s3://lamindb-ci/test-anndata")

We'll need some test data:

In [ ]:
ln.Artifact("s3://lamindb-ci/test-anndata/pbmc68k.h5ad").save()

An `h5ad` artifact stored on s3:

In [ ]:
artifact = ln.Artifact.filter(key="pbmc68k.h5ad").one()

In [ ]:
artifact.path

In [ ]:
adata = artifact.open()

In [ ]:
adata

It is possible to access `AnnData` attributes without loading them into memory

In [ ]:
print(adata.obsm)
print(adata.varm)
print(adata.obsp)
print(adata.varm)

However, `.obs`, `.var` and `.uns` are always loaded fully into memory on `AnnDataAccessor` initialization

In [ ]:
adata.obs.head()

In [ ]:
adata.var.head()

In [ ]:
adata.uns.keys()

Without subsetting, the `AnnDataAccessor` object gives references to underlying lazy `h5` or `zarr` arrays:

In [ ]:
adata.X

In [ ]:
adata.obsm["X_pca"]

And to a lazy `SparseDataset` from the `anndata` package:

In [ ]:
adata.obsp["distances"]

Get a subset of the object, attributes are loaded only on explicit access:

In [ ]:
obs_idx = adata.obs.cell_type.isin(["Dendritic cells", "CD14+ Monocytes"]) & (
    adata.obs.percent_mito <= 0.05
)
adata_subset = adata[obs_idx]

In [ ]:
adata_subset

Check shapes of the subset

In [ ]:
num_idx = sum(obs_idx)
assert adata_subset.shape == (num_idx, adata.shape[1])
assert (adata_subset.obs.cell_type == "CD34+").sum() == 0

In [ ]:
adata_subset.obs.cell_type.value_counts()

Subsets load the arrays into memory only on direct access

In [ ]:
print(adata_subset.X)

In [ ]:
print(adata_subset.obsm["X_pca"])

In [ ]:
assert adata_subset.obsp["distances"].shape[0] == num_idx

To load the entire subset into memory as an actual `AnnData` object, use `to_memory()`:

In [ ]:
adata_subset.to_memory()

In [ ]:
!lamin delete --force test-anndata